# Transfer Learning Assignment: Inception V3
## Lilah Mercadante, April 6, 2023


I chose to use Inception V3 to classify images from a flower dataset. The data consists of 924 photos of five different types of flowers: Daisy, sunflower, tulip, rose, and dandelion. 

In [ ]:
# imports and setting up model
# I chose to use Inception version 3

import numpy as np
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 136MB/s] 


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
pwd

'/content'

In [ ]:
# import required module
import os
from PIL import Image
from torchvision import transforms
# assign directory
directory = '/content/drive/MyDrive/Colab Notebooks/archive/test/'

In [ ]:
test_imags = []
for file in os.listdir(directory): 
  filename = '/content/drive/MyDrive/Colab Notebooks/archive/test/' + file
  input_image = Image.open(filename)
  preprocess = transforms.Compose([
      transforms.Resize(299),
      transforms.CenterCrop(299),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      ])
  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
  test_imags.append(input_batch)

In [ ]:
probabilities = []
filenames = []
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
    # checking if it is a file
  if os.path.isfile(f):
      input_image = Image.open(f)
      preprocess = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
      input_tensor = preprocess(input_image)
      input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

      # move the input and model to GPU for speed if available
      if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

      with torch.no_grad():
        output = model(input_batch)
        # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
        #print(output[0])
        # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
        probability = torch.nn.functional.softmax(output[0], dim=0)
        probabilities.append(probability)
        filenames.append(filename)
        #print(probabilities)

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-04-07 18:13:20--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-04-07 18:13:20 (89.6 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
for prob in probabilities:
  top5_prob, top5_catid = torch.topk(prob, 1)
  for i in range(top5_prob.size(0)):
      print("Category is = ", categories[top5_catid[i]], top5_prob[i].item())

Category is =  daisy 0.21113945543766022
Category is =  daisy 0.9234874248504639
Category is =  daisy 0.9176577925682068
Category is =  bee 0.4471687376499176
Category is =  daisy 0.8659324645996094
Category is =  daisy 0.8497336506843567
Category is =  daisy 0.17282608151435852
Category is =  spider web 0.29541200399398804
Category is =  miniskirt 0.07544340938329697
Category is =  vase 0.2813461422920227
Category is =  daisy 0.21053621172904968
Category is =  daisy 0.9379180669784546
Category is =  feather boa 0.3681274950504303
Category is =  cardoon 0.5984128713607788
Category is =  daisy 0.902233898639679
Category is =  daisy 0.9502587914466858
Category is =  picket fence 0.30443981289863586
Category is =  mushroom 0.8341621160507202
Category is =  jackfruit 0.3219676613807678
Category is =  dishwasher 0.18678532540798187
Category is =  cabbage butterfly 0.1999969482421875
Category is =  hair slide 0.17552794516086578
Category is =  daisy 0.6904805898666382
Category is =  picket f

## Comments:

I can see that Inception V3 does pretty well at identifying daisies, however it cannot tell the difference between a daisy and another flower. Also for some flowers it does not recognize them as a flower at all. 